## Brain Tumor classification: Quantum ML

This solution provides a hybrid classical-quantum trainable ML pipeline for brain tumor classification using brain MRI images.


This sample notebook shows you how to build brain tumor classifier using hybrid classical-quantum model.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Vector Search for Company Description. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure the dataset](#B.-Configure-the-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train a machine learning model](#3:-Train-a-machine-learning-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-a-model)
1. [Deploy model and verify results](#4:-Deploy-model-and-verify-results)
    1. [Deploy trained model](#A.-Deploy-trained-model)
    1. [Create input payload](#B.-Create-input-payload)
    1. [Perform real-time inference](#C.-Perform-real-time-inference)
    1. [Visualize output](#D.-Visualize-output)
    1. [Delete the endpoint](#E.-Delete-the-endpoint)
1. [Perform Batch inference](#5.-Perform-Batch-inference)
    1. [Run batch-transform job](#A.-Run-the-batch-transform-job)
    1. [Inspect the Output](#B.-Inspect-the-Batch-Transform-Output-in-S3)
1. [Clean-up](#6.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page **Brain Tumor classification: Quantum ML**
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [36]:
algo_arn ='brain-tumor-classification-v2'

### 2. Prepare dataset

In [37]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from urllib.parse import urlparse
import boto3
import urllib.request
import numpy as np
from zipfile import ZipFile
import pandas as pd

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described for best results:
* The training pipeline need two files data.zip and parameters.json.
* The data.zip contains MRI images of different tumor types.
* parameters.json contains parameters of training pipeline.

#### B. Configure the dataset

In [3]:
training_dataset='input/data.zip'

#### C. Upload datasets to Amazon S3

In [4]:
sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()

In [ ]:
# training input location
common_prefix = "brain-tumor-classification"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "input"
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix)
print("Training input uploaded to " + training_input)

## 3: Train a machine learning model

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [6]:
role = get_execution_role()

In [7]:
output_location = 's3://{}/brain-tumor-classification/{}'.format(bucket, 'output')

### 3.2 Train a model

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

Please select an appropriate instance type based on the training dataset size.  

In [8]:
instance_type='ml.m5.4xlarge'

In [12]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="brain-tumor-classification-training",
    role=role,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=instance_type
)
#Run the training job.
estimator.fit({"training": training_input})

2023-08-17 11:10:23 Starting - Starting the training job...
2023-08-17 11:10:36 Starting - Preparing the instances for trainingProfilerReport-1692270622: InProgress
......
2023-08-17 11:11:50 Downloading - Downloading input data
2023-08-17 11:11:50 Training - Downloading the training image...
2023-08-17 11:12:10 Training - Training image download completed. Training in progress.2023-08-17 11:12:14.615698: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-17 11:12:14.663680: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-17 11:12:14.664195: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 11:12:15.337585:

2023-08-17 11:17:21.981364: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
#011 [[{{node Placeholder/_0}}]]
#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 96/161 [================>.............] - ETA: 2:02 - loss: 0.0158 - accuracy: 0.9987#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 97/161 [=================>..


2023-08-17 11:18:06 Uploading - Uploading generated training model
2023-08-17 11:18:06 Completed - Training job completed
Training seconds: 398
Billable seconds: 398


See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

### 4: Deploy model and verify results

Now you can deploy the model for performing real-time inference.

In [14]:
model_name='brain-tumor-classification'

content_type='application/zip'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Deploy trained model

In [15]:
from sagemaker.predictor import csv_serializer
predictor = estimator.deploy(1, real_time_inference_instance_type)

..........
----!

Once endpoint is created, you can perform real-time inference.

#### B. Create input payload

The trained model accepts zip file containing brain MRI images.  
For detailed instructions, please refer input details.

In [16]:
file_name = 'inference_input/data_inference.zip'

#### C. Perform real-time inference

In [19]:
output_file_name = 'output.zip'

In [20]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $predictor.endpoint_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    $output_file_name

{
    "ContentType": "application/zip",
    "InvokedProductionVariant": "AllTraffic"
}


In [24]:
# unzipping output.zip in inference_output folder 
from zipfile import ZipFile
with ZipFile('output.zip', 'r') as zObject:
    zObject.extractall('inference_output')

#### D. Visualize output

The model output is saved in inference_output folder which contains images named in the following format: orginal_images_name-predicted_class_name.jpg.

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. you can terminate the same to avoid being charged.

In [25]:
predictor.delete_endpoint(delete_endpoint_config=True)

### 5. Perform Batch inference

In this section, you will perform batch inference using multiple input payloads together.

#### A. Run the batch-transform job

In [ ]:
#upload the batch-transform job input files to S3
transform_input_folder = "inference_input/data_inference.zip"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

In [29]:
#Run the batch-transform job
transformer = estimator.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

..........
.........................2023-08-17 12:28:06.422255: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
 * Serving Flask app 'serve'
 * Debug mode: off
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://169.254.255.131:8080
#033Press CTRL+C to quit#033
2023-08-17 12:28:06.422255: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
 * Serving Flask app 'serve'
 * Debug mode: off
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://169.254.255.131:8080
#033Press CTRL+C to quit#033
169.254.255.130 - - [17/Aug/2023 12:28:12] "GET /ping HTTP/1.1" 200 

#0151/1 [==============================] - ETA: 0s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151/1 [==============================] - 0s 80ms/step
#0151/1 [==============================] - ETA: 0s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151/1 [==============================] - 0s 85ms/step
169.254.255.130 - - [17/Aug/2023 12:28:14] "POST /invocations HTTP/1.1" 200 -


In [ ]:
#output is available on following path
transformer.output_path

#### B. Inspect the Batch Transform Output in S3

In [33]:
import os
s3_conn = boto3.client("s3")
with open('output.zip', 'wb') as f:
    s3_conn.download_fileobj(bucket, os.path.basename(transformer.output_path)+'/data_inference.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [34]:
# unzipping output.zip
with ZipFile('output.zip', 'r') as zObject:
    zObject.extractall('inference_output')

### 6. Clean-up

#### A. Delete the model

In [35]:
transformer.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

